In [506]:
import pandas as pd
import random
import io
import matplotlib.pyplot as plt
import sys
import os
import codecs
import string
import re
from ipywidgets import IntProgress
from IPython.display import display
from tqdm.auto import tqdm
from ordered_set import OrderedSet
from nltk import edit_distance
from nltk.metrics.distance import jaro_similarity

Гиперпараметры:

In [507]:
min_score_limit = 0.5
vendor_filter = False

Функция поиска оптимального соответствия:

In [508]:
def match(pattern, options):
    
    best_match = None
    min_score = min_score_limit
    m_vendor = d_vendor = '1'
    
    for item in options:
        
        if vendor_filter is True: 
            m_vendor = df_market[df_market.model_m==pattern].ven_m
            d_vendor = df_dev[df_dev.model_d==item].ven_d
        
        if str(m_vendor)==str(d_vendor):
            
            p_set = OrderedSet(pattern.split())
            i_set = OrderedSet(item.split())
            
            jaro_score = jaro_similarity(p_set, i_set)
            jaccard_score = len(p_set.intersection(i_set))/len(p_set.union(i_set))
            levenstein_score = edit_distance(p_set, i_set) 
            
            current_score = levenstein_score
            
            if  current_score > min_score:
                current_score = min_score
                best_match = item

    return best_match

Читаем очищенный каталог устройств в df_devices:

In [509]:
df_dev = pd.read_csv('data/devices_clean.csv') 
df_dev = df_devices.astype({'ven_d': 'str', 'model_d': 'str', 'tac_d': 'str'})
df_dev.head(0)

,ven_d,model_d,tac_d


Читаем очищенный каталог ЯМ-TOP в 'df_ym_top':

In [510]:
df_market = pd.read_csv('data/market_clean.csv')
df_market = df_market.astype({'ven_m': 'str', 'model_m': 'str', 'class_m': 'str'})
df_market = df_market[['ven_m', 'model_m', 'class_m']]
df_market.head(0)

,ven_m,model_m,class_m


Сортировка по названию производителя:

In [511]:
df_market.sort_values('ven_m', axis = 0, ascending = True, inplace = True) 
df_dev.sort_values('ven_d', axis = 0, ascending = True, inplace = True) 

Ищем соответствия:

In [512]:
tqdm.pandas()
df_market['match_d'] = df_market['model_m'].progress_apply(lambda x: match(str(x), df_dev['model_d']))
df_market_top = df_market_top[['class_m', 'ven_m', 'model_m', 'match']]

Выводим случайный сэмпл:

In [513]:
df_market_top.sample(n=10)

,class_m,ven_m,model_m,match
6484,видеорегистраторы,street storm,c a7810,None
1767,видеорегистраторы,cansonic,707 duo,None
3170,телевизоры,lg,49um7450,None
587,телевизоры,samsung,ue40h6200,None
5330,gps навигаторы,ibox,pro 7500 hd,None
1123,радиотелефоны,gigaset,e310,None
3023,планшеты,asus,tab note 8 m80ta 64gb,None
7019,телевизоры,mystery,mtv 3223lt2,None
2084,планшеты,samsung,galaxy view 18 4 sm t677 32gb,None
5807,автомагнитолы,pioneer,sph da01,None
